In [ ]:
import json
import requests
import subprocess

### Login security checks <a class="anchor" id="head-1.2"></a>

In [ ]:
output = subprocess.getoutput("curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"1\"}'")
print(output)
# curl: (28) Failed to connect to sqa-tao.metropolis.nvidia.com port 32080: Connection timed out

In [ ]:
output = subprocess.getoutput("curl -X POST http://sqa-tao.metropolis.nvidia.com:32443/api/v1/login -d '{\"ngc_api_key\": \"1\"}'")
print(output)
# 302 Found HTML body

In [ ]:
output = subprocess.getoutput("curl -X POST https://sqa-tao.metropolis.nvidia.com:32443/api/v1/login -d '{\"ngc_api_key\": \"1\"}'")
print(output)
# Empty json

In [ ]:
output = subprocess.getoutput("curl -X POST https://35.86.158.141:32443/api/v1/login -d '{\"ngc_api_key\": \"1\"}'")
print(output)
# SSL certificate problem

### FIXME

1. Assign the ip_address and port_number for user 1 in FIXME 1 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_api_key variable for user 1 in FIXME 2
1. Assign the ngc_api_key variable for user 2 in FIXME 3

In [ ]:
host_url = "http://<ip_address>:<port_number>" # FIXME1 example: https://10.137.149.22:32334
# In host machine, node ip_address and port number can be obtained as follows,
# ip_address: hostname -i
# port_number: kubectl get service ingress-nginx-controller -o jsonpath='{.spec.ports[0].nodePort}'
ngc_api_key_user_1 = "<ngc_api_key_1>" # FIXME2 example: (Add NGC API key)
ngc_api_key_user_2 = "<ngc_api_key_2>" # FIXME3 example: (Add NGC API key)

### Login for user_1 <a class="anchor" id="head-1.2"></a>

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key_user_1})
response = requests.post(f"{host_url}/api/v1/login", data=data)
assert response.status_code in (200, 201)
assert "user_id" in response.json().keys()
user_id_user_1 = response.json()["user_id"]
print("User ID",user_id_user_1)
assert "token" in response.json().keys()
token_user_1 = response.json()["token"]
print("JWT",token_user_1)

# Set base URL
base_url_user_1 = f"{host_url}/api/v1/user/{user_id_user_1}"
print("API Calls will be forwarded to",base_url_user_1)

headers_user_1 = {"Authorization": f"Bearer {token_user_1}"}

### Create dataset for user_1 <a class="anchor" id="head-1.2"></a>

In [ ]:
# Create dataset
data = json.dumps({"type":"semantic_segmentation","format":"unet"})

endpoint = f"{base_url_user_1}/dataset"

response = requests.post(endpoint,data=data,headers=headers_user_1)
assert response.status_code in (200, 201)

print(response)
print(response.json())

assert "id" in response.json().keys()
dataset_id_user_1 = response.json()["id"]

### Create model for user_1 <a class="anchor" id="head-4"></a>

In [ ]:
data = json.dumps({"network_arch":"unet","encryption_key":"tlt_encode","checkpoint_choose_method":"best_model"})

endpoint = f"{base_url_user_1}/model"

response = requests.post(endpoint,data=data,headers=headers_user_1)
assert response.status_code in (200, 201)

print(response)
print(response.json())
assert "id" in response.json().keys()
model_id_user_1 = response.json()["id"]

### Login for user_2 <a class="anchor" id="head-1.2"></a>

In [ ]:
# Exchange NGC_API_KEY for JWT
data = json.dumps({"ngc_api_key": ngc_api_key_user_2})
response = requests.post(f"{host_url}/api/v1/login", data=data)
print(response)
assert response.status_code in (200, 201)
assert "user_id" in response.json().keys()
user_id_user_2 = response.json()["user_id"]
print("User ID",user_id_user_2)
assert "token" in response.json().keys()
token_user_2 = response.json()["token"]
print("JWT",token_user_2)

# Set base URL
base_url_user_2 = f"{host_url}/api/v1/user/{user_id_user_2}"
print("API Calls will be forwarded to",base_url_user_2)

headers_user_2 = {"Authorization": f"Bearer {token_user_2}"}

### Create dataset for user_2 <a class="anchor" id="head-1.2"></a>

In [ ]:
# Create dataset
data = json.dumps({"type":"instance_segmentation","format":"coco"})

endpoint = f"{base_url_user_2}/dataset"

response = requests.post(endpoint,data=data,headers=headers_user_2)
assert response.status_code in (200, 201)

print(response)
print(response.json())

assert "id" in response.json().keys()
dataset_id_user_2 = response.json()["id"]

### Create model for user_2 <a class="anchor" id="head-4"></a>

In [ ]:
data = json.dumps({"network_arch":"mask_rcnn","encryption_key":"tlt_encode","checkpoint_choose_method":"best_model"})

endpoint = f"{base_url_user_2}/model"

response = requests.post(endpoint,data=data,headers=headers_user_2)
assert response.status_code in (200, 201)

print(response)
print(response.json())
assert "id" in response.json().keys()
model_id_user_2 = response.json()["id"]

### Dataset access security checks <a class="anchor" id="head-4"></a>

#### UC - 5 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/dataset/{dataset_id_user_2}')
print(output)
# should return: Dataset not found

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_1}')
print(output)
# should return: Dataset not found

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/dataset/{dataset_id_user_1}')
print(output)
# Returns dataset metadata (format:unet, type:semantic segmentation)

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# Returns dataset metadata (format:coco, type:instance segmentation)

### UC - 6 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f"curl https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/dataset/{dataset_id_user_1}")
print(output)
# Returns authorization required HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# Returns Forbidden HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/dataset/{dataset_id_user_1}')
print(output)
# Returns Forbidden HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# Returns dataset metadata (format:coco, type:instance segmentation)

#### UC - 7 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput("curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"anyRandomString\"}")
print(output)
# curl: (28) Failed to connect to sqa-tao.metropolis.nvidia.com port 32080: Connection timed out

In [ ]:
output = subprocess.getoutput("curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"YzZtczM5amdtdDcwNjk0cnA2bGU2bXQ3bnQ6NmQ4NjNhMDItMTdmZS00Y2QxLWI2ZjktNmE5M2YxZTc0OGMv\"}")
print(output)
# curl: (28) Failed to connect to sqa-tao.metropolis.nvidia.com port 32080: Connection timed out

In [ ]:
output = subprocess.getoutput(f"curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"{ngc_api_key_user_1}\"}'")
print(output)

#### UC - 8 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f"curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"{ngc_api_key_user_1}\"}")
print(output)

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# should return dataset dataset_id_user_2 metadata
# ask admin to check app pod logs, authentication was checked against cached user session

In [ ]:
# wait over 2 hours with no activity for user user_id_user_2
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# should return dataset dataset_id_user_2 metadata
# ask admin to check app pod logs, authentication was checked against cached user session, failed, then authenticated the user again against NVIDIA servers

#### UC - 9 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/dataset/{dataset_id_user_2}')
print(output)
# should return dataset dataset_id_user_2 metadata
# ask admin to check app pod logs, authentication was checked against cached user session
# repeat the above a few times within 2 hours

### Model access security checks <a class="anchor" id="head-4"></a>

#### UC - 5 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/model/{model_id_user_2}')
print(output)
# should return: Model not found

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_1}')
print(output)
# should return: Model not found

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/model/{model_id_user_1}')
print(output)
# Returns model metadata (network_arch:unet)

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# Returns dataset metadata (network_arch:mask_rcnn)

### UC - 6 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f"curl https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/model/{model_id_user_1}")
print(output)
# Returns authorization required HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_1}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# Returns Forbidden HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_1}/model/{model_id_user_1}')
print(output)
# Returns Forbidden HTML body

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# Returns model metadata (network_arch:mask_rcnn, type:instance segmentation)

#### UC - 8 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f"curl -X POST http://sqa-tao.metropolis.nvidia.com:32080/api/v1/login -d '{\"ngc_api_key\": \"{ngc_api_key_user_1}\"}")
print(output)

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# should return model model metadata
# ask admin to check app pod logs, authentication was checked against cached user session

In [ ]:
# wait over 2 hours with no activity for user user_id_user_2
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# should return model model_id_user_2 metadata
# ask admin to check app pod logs, authentication was checked against cached user session, failed, then authenticated the user again against NVIDIA servers

#### UC - 9 <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput(f'curl -H "Authorization: Bearer {token_user_2}" https://sqa-tao.metropolis.nvidia.com:32443/api/v1/user/{user_id_user_2}/model/{model_id_user_2}')
print(output)
# should return model model_id_user_2 metadata
# ask admin to check app pod logs, authentication was checked against cached user session
# repeat the above a few times within 2 hours

### Cluster access check <a class="anchor" id="head-4"></a>

In [ ]:
output = subprocess.getoutput('ssh sqa-tao.metropolis.nvidia.com')
print(output)